# Name-Based Kernel Inspection \& Filtering

In this notebook you will learn how to:

* Query different types of kernel names
* Use names to filter kernels within a report

## Setup

First, import NVIDIA Nsight Compute's Python Report Interface (PRI) as `ncu_report`
and load an `ncu-rep` report file with `load_report`:

In [ ]:
import ncu_report

report_file_path = "../sample_reports/mergeSort.ncu-rep"
report = ncu_report.load_report(report_file_path)

For later use, unpack the profiling results for all `kernels` contained in the first range:

In [ ]:
kernels = report[0]

## Types of Kernel (`IAction`) Names

The `IRange` object `kernels` contains a number of `IAction` objects, each of which represents the
profiling result of an individual kernel.

To get an overview of the contents of your profiling report you may first want to iterate over the `IAction` objects
in `kernels`.
`IAction` objects are convertible to strings, which will implicitly call their member function `name()`:

In [ ]:
for index, kernel in enumerate(kernels):
    print(f"{index}: {kernel}")

Apart from querying the _function name_ of a kernel, you can also use `IAction.name()` to query the kernel's
_mangled_ and _demangled name_. While the former is generated by the compiler and used in the compiled binary,
the latter is essentially the function's signature.

In order to obtain the mangled and demangled names of a kernel, you need to pass the additional enum values
`IAction.NameBase_MANGLED` and `IAction.NameBase_DEMANGLED` to `name`, respectively. Here's an example:

In [ ]:
kernel = kernels[0]
print(f"{kernel}'s demangled name: {kernel.name(ncu_report.IAction.NameBase_MANGLED)}")
print(f"{kernel}'s signature: {kernel.name(ncu_report.IAction.NameBase_DEMANGLED)}")

## Name-Based Kernel Filtering

Since NVIDIA Nsight Compute reports may contain profiling results for different kernel
implementation, it may sometimes be useful to filter results based on kernel names.

As an example, you can use `name()` to find all unique kernel implementations contained within a report:

In [ ]:
unique_kernels = {}

for kernel in kernels:
    name = kernel.name(ncu_report.IAction.NameBase_FUNCTION)
    mangled_name = kernel.name(ncu_report.IAction.NameBase_MANGLED)

    if name not in unique_kernels:
        unique_kernels[name] = mangled_name


print("Unique kernels found in report:")
for name, mangled_name in unique_kernels.items():
    print(f"  \033[1m{name}\033[0m (mangled: {mangled_name})")

Name-based filtering may also be useful when aggregating metrics over all profiling results of a given kernel.

Here's an example, illustrating how filtering can be used to get a basic understand of the scattering
of the throughput of one of the above kernels:

In [ ]:
kernel_name = "mergeElementaryIntervalsKernel"
metric_name = "sm__throughput.avg.pct_of_peak_sustained_elapsed"
throughputs = []

for kernel in kernels:
    if str(kernel) == kernel_name:
        throughputs.append(kernel[metric_name].value())


print(
    f"Compute (SM) Throughput for Kernel \033[1m{kernel_name}\033[0m:\n"
    f"  average = {sum(throughputs)/len(throughputs):.2f}%, "
    f"minimum = {min(throughputs):.2f}%, "
    f"maximum = {max(throughputs):.2f}%"
)